# initial setup

In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
!pip3 install ipywidgets --user

In [ ]:
!pip install -r yolov5/requirements.txt

In [ ]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2

# load model

In [ ]:
model= torch.hub.load('ultralytics/yolov5','yolov5s')


In [ ]:
results=model('https://www.businessinsider.in/thumb/msid-77876393,width-700,resizemode-4,imgsize-327845/science/news/23-bus-passengers-contracted-coronavirus-from-1-infected-person-no-one-was-wearing-a-mask-/img5f4d79597ffa48002894d46a.jpg')

In [ ]:
results.show()

# real time detection

In [ ]:
cam=cv2.VideoCapture(0)
while cam.isOpened():
    ret,frame=cam.read()
    results=model(frame)
    
    cv2.imshow("Yolo",np.squeeze(results.render()))
    
    k=cv2.waitKey(10)
    if k == ord('q'):
        break
cam.release()
cv2.destroyAllWindows()

# train from scratch

In [ ]:
!cd yolov5 && python train.py --batch 16 --epochs 90 --data ../data.yaml --weights yolov5s.pt

# load Model

In [1]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
import os

In [2]:
model= torch.hub.load('ultralytics/yolov5','custom',path='yolov5/runs/train/exp3/weights/last.pt')

Using cache found in C:\Users\kuber/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-11-13 Python-3.10.6 torch-1.13.0 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [3]:
import easyocr
reader=easyocr.Reader(['en'])

In [4]:
di='test/images'
files=os.listdir(di)
print(files)

['0.jpg', '1.jpg', '2.jpg', '3.jpg', '4.JPG', 'Cars115_png.rf.048d36d6a6f34308dbad3fbd2bf9f5e8.jpg', 'Cars118_png.rf.e8500c6c26b4a6bc3615ec83ebdd41b1.jpg', 'Cars19_png.rf.be8f69e48132b4f9581ee417bb10b7fe.jpg', 'Cars205_png.rf.d03dc59ee3076c5115659b03cd1d27ef.jpg', 'Cars220_png.rf.3d9b32e89e26847390b40889b1ec0b7e.jpg', 'Cars226_png.rf.19c1afa2f02dc782ea8c18883952831d.jpg', 'Cars264_png.rf.e84db51a76517ff40332279a270a62de.jpg', 'Cars265_png.rf.d5634ffb3dde27962e154769508ecbf8.jpg', 'Cars278_png.rf.2029f70d64b05e4569185da86c3c537e.jpg', 'Cars37_png.rf.0017a4edf9665e20b6c30b0ac771c274.jpg']


In [6]:
def binary(img):
    gray_image=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #thresh,im_bw=cv2.threshold(gray_image,100,200,cv2.THRESH_BINARY)
    return im_bw

In [7]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd='C:\\Users\\kuber\\Study material\\research\\install\\tesseract\\tesseract.exe'

In [8]:
def ocr_core(img):
    text= pytesseract.image_to_string(img)
    return text

In [9]:
def get_grayscale(image):
    return cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

In [10]:
def remove_noise(image):
    return cv2.medianBlur(image,5)

In [11]:
def threshold(image):
    return cv2.threshold(image,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]

In [12]:
import numpy as np
from skimage.transform import hough_line, hough_line_peaks
from skimage.transform import rotate
from skimage.feature import canny
from skimage.io import imread
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
from scipy.stats import mode

In [13]:
def line_img(image):
    edges = canny(image)
    # Classic straight-line Hough transform
    tested_angles = np.deg2rad(np.arange(0.1, 180.0))
    h, theta, d = hough_line(edges, theta=tested_angles)

    # Generating figure 1
    fig, axes = plt.subplots(1, 2, figsize=(15, 16))
    ax = axes.ravel()

    ax[0].imshow(image, cmap="gray")
    ax[0].set_title('Input image')
    ax[0].set_axis_off()

    ax[1].imshow(edges, cmap="gray")
    origin = np.array((0, image.shape[1]))

    for _, angle, dist in zip(*hough_line_peaks(h, theta, d)):
        y0, y1 = (dist - origin * np.cos(angle)) / np.sin(angle)
        ax[1].plot(origin, (y0, y1), '-r')

    ax[1].set_xlim(origin)
    ax[1].set_ylim((edges.shape[0], 0))
    ax[1].set_axis_off()
    ax[1].set_title('Detected lines')

In [14]:
def skew_angle_hough_transform(image):
    # convert to edges
    edges = canny(image)
    # Classic straight-line Hough transform between 0.1 - 180 degrees.
    tested_angles = np.deg2rad(np.arange(0.1, 180.0))
    h, theta, d = hough_line(edges, theta=tested_angles)
    # find line peaks and angles
    accum, angles, dists = hough_line_peaks(h, theta, d)
    
    # round the angles to 2 decimal places and find the most common angle.
    most_common_angle = mode(np.around(angles, decimals=2))[0]
    
    # convert the angle to degree for rotation.
    skew_angle = np.rad2deg(most_common_angle - np.pi/2)
    return skew_angle

In [15]:
def rotation(image,angle):
    image_rotation=cv2.warpAffine(image,cv2.getRotationMatrix2D((0,0),int(angle),1),(0,0))
    return image_rotation

In [16]:
def get_image(img):
    results=model(img)
    bbox=results.xyxy[0]
    bound=[]
    for j in bbox:
        temp=[]
        for k in j:
            temp.append(int(k.item()))
        bound.append(temp)
    for j in bound:
        temp=img[j[1]:j[3],j[0]:j[2]]
        temp=get_grayscale(temp)
#         line_img(temp)
        angle=skew_angle_hough_transform(temp)
        temp=rotation(temp,angle)
        result=reader.readtext(temp)
        if(len(result)>0):
            top_left= tuple([int(k) for k in result[-1][0][0]])
            bottom_right= tuple([int(k) for k in result[-1][0][2]])
            text = result[-1][1]
            font = cv2.FONT_HERSHEY_SIMPLEX
            img= cv2.rectangle(img,(j[0],j[1]),(j[2],j[3]),(0,255,0),5)
            img= cv2.putText(img,text,(j[0],j[1]),font,.5,(255,255,255),2,cv2.LINE_AA)
    return img
        
        
    

In [18]:
%matplotlib inline
for i in files:
    img=os.path.join(di,i)
    img=cv2.imread(img)
    temp=get_grayscale(img)
    results=model(temp)
    bbox=results.xyxy[0]
    bound=[]
    for j in bbox:
        temp=[]
        for k in j:
            temp.append(int(k.item()))
        bound.append(temp)
    for j in bound:
        temp=img[j[1]:j[3],j[0]:j[2]]
        temp=get_grayscale(temp)
#         line_img(temp)
        angle=skew_angle_hough_transform(temp)
        temp=rotation(temp,angle)
        result=reader.readtext(temp)
        if(len(result)>0):
            top_left= tuple([int(k) for k in result[-1][0][0]])
            bottom_right= tuple([int(k) for k in result[-1][0][2]])
            text = result[-1][1]
            font = cv2.FONT_HERSHEY_SIMPLEX
            temp= cv2.rectangle(img,(j[0],j[1]),(j[2],j[3]),(0,255,0),5)
            temp= cv2.putText(img,text,(j[0],j[1]),font,.5,(255,255,255),2,cv2.LINE_AA)
        cv2.imshow("",temp)
        cv2.waitKey(0)
cv2.destroyAllWindows()
        
    

C:\Users\kuber\AppData\Local\Temp\ipykernel_26052\2873010673.py:11: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  most_common_angle = mode(np.around(angles, decimals=2))[0]
C:\Users\kuber\AppData\Local\Temp\ipykernel_26052\2873010673.py:11: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to

# live capture

In [129]:
cap=cv2.VideoCapture(0)
while(1):
    _,frame=cap.read()
    frame=get_image(frame)
    cv2.imshow("",frame)
    if cv2.waitKey(1000) & 0xFF==ord('q'):
        break
cv2.destroyAllWindows()
cap.release()
    

C:\Users\kuber\AppData\Local\Temp\ipykernel_11124\2873010673.py:11: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  most_common_angle = mode(np.around(angles, decimals=2))[0]
C:\Users\kuber\AppData\Local\Temp\ipykernel_11124\2873010673.py:11: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to

C:\Users\kuber\AppData\Local\Temp\ipykernel_11124\2873010673.py:11: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  most_common_angle = mode(np.around(angles, decimals=2))[0]
C:\Users\kuber\AppData\Local\Temp\ipykernel_11124\2873010673.py:11: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to

C:\Users\kuber\AppData\Local\Temp\ipykernel_11124\2873010673.py:11: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  most_common_angle = mode(np.around(angles, decimals=2))[0]
C:\Users\kuber\AppData\Local\Temp\ipykernel_11124\2873010673.py:11: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to

C:\Users\kuber\AppData\Local\Temp\ipykernel_11124\2873010673.py:11: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  most_common_angle = mode(np.around(angles, decimals=2))[0]
C:\Users\kuber\AppData\Local\Temp\ipykernel_11124\2873010673.py:11: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to

C:\Users\kuber\AppData\Local\Temp\ipykernel_11124\2873010673.py:11: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  most_common_angle = mode(np.around(angles, decimals=2))[0]
C:\Users\kuber\AppData\Local\Temp\ipykernel_11124\2873010673.py:11: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to

C:\Users\kuber\AppData\Local\Temp\ipykernel_11124\2873010673.py:11: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  most_common_angle = mode(np.around(angles, decimals=2))[0]
C:\Users\kuber\AppData\Local\Temp\ipykernel_11124\2873010673.py:11: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to

C:\Users\kuber\AppData\Local\Temp\ipykernel_11124\2873010673.py:11: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  most_common_angle = mode(np.around(angles, decimals=2))[0]
C:\Users\kuber\AppData\Local\Temp\ipykernel_11124\2873010673.py:11: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to

C:\Users\kuber\AppData\Local\Temp\ipykernel_11124\2873010673.py:11: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  most_common_angle = mode(np.around(angles, decimals=2))[0]
C:\Users\kuber\AppData\Local\Temp\ipykernel_11124\2873010673.py:11: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to

C:\Users\kuber\AppData\Local\Temp\ipykernel_11124\2873010673.py:11: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  most_common_angle = mode(np.around(angles, decimals=2))[0]
C:\Users\kuber\AppData\Local\Temp\ipykernel_11124\2873010673.py:11: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to